In [ ]:
#Upper Part, obtainning data from binance, have to run seperatly

import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

api_key = 'YOV8V5EkOCoAoYZbkj8HaOIGuWVZP6AIeV0z2a7wqFwFBxUrSBy4dVRFxlvy8YmA'
api_secret = 'zXguUnLvbfbum7NWQCwCpnQ4OU3h5qwHbuRrgackWw0e5ZGpFAf5US1QqpLswxRd'
client = Client(api_key,api_secret)
bsm = BinanceSocketManager(client) 
socket = bsm.trade_socket('BTCUSDT')

engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

def createframe(msg): #transferring the msg to a dataframe
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']] #we just want the symbol, event time, and price
    df.columns = ['Symbol','Time','Price']
    df.Price = df.Price.astype(float) #becuz it was string type
    df.Time = pd.to_datetime(df.Time, unit='ms') #becuz it was standard time
    return df

while True:
    await socket.__aenter__()
    msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
    print(frame)

In [6]:
#Lower part, the trading bot, have to run seperatly
import sqlalchemy
import pandas as pd
from binance.client import Client

api_key = 'YOV8V5EkOCoAoYZbkj8HaOIGuWVZP6AIeV0z2a7wqFwFBxUrSBy4dVRFxlvy8YmA'
api_secret = 'zXguUnLvbfbum7NWQCwCpnQ4OU3h5qwHbuRrgackWw0e5ZGpFAf5US1QqpLswxRd'
client = Client(api_key,api_secret)

engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

def ptrade(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql('BTCUSDT', engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change()+1).cumprod()-1 
        if not open_position: 
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol='BTCUSDT',side='BUY',type='MARKET',quantity=qty)
                cost = float(order['fills'][0]['price'])*qty
                commission1 = float(order['fills'][0]['commission'])
                print(f'Entry Price of the trade: {order["fills"][0]["price"]}')
                print(f'Cost of transactions: ${cost}USDT')
                open_position = True 
                break
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time>pd.to_datetime(order['transactTime'],unit='ms')]
            
            if len(sincebuy)>1: 
                sincebuyret = (sincebuy.Price.pct_change()+1).cumprod()-1
                last_entry = sincebuyret[sincebuyret.last_valid_index()] 
                if last_entry>0.0015 or last_entry<-0.0015:
                    if qty-commission1>0.001:
                        amount = qty-commission1
                    else:
                        amount = 0.001 #minimum amount
                    order = client.create_order(symbol='BTCUSDT',side='SELL',type='MARKET',quantity=float(amount))
                    if len(order)==14:
                        profit=0
                        for n in range(len(order["fills"])):
                            profit += (float(order["fills"][n]["qty"])*float(order["fills"][n]["price"]))-float(order["fills"][n]["commission"])
                        profit -= cost
                        print(f'Exit price of the trade: {order["fills"][0]["price"]}')
                        print(f'gain/loss for this trade: ${profit}USTD')
                    break
while True:
    ptrade(0.001,60,0.001)

Entry Price of the trade: 38666.29000000
Cost of transactions: $38.666290000000004USDT
Exit price of the trade: 38608.66000000
gain/loss for this trade: $-0.09623865999999737USTD
Entry Price of the trade: 38544.00000000
Cost of transactions: $38.544000000000004USDT
Exit price of the trade: 38599.01000000
gain/loss for this trade: $0.01640617000000333USTD
Entry Price of the trade: 38598.99000000
Cost of transactions: $38.59899USDT
Exit price of the trade: 38540.00000000
gain/loss for this trade: $-0.097529999999999USTD
Entry Price of the trade: 38562.87000000
Cost of transactions: $38.562870000000004USDT


KeyboardInterrupt: 

In [ ]:
#死因: the database end up getting too full
#improvement: clear up the database regularly